# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os
import codecs

In [13]:
replacement = {
   '85' : '...',           # u'\u2026' ... character.
   '96' : '-',             # u'\u2013' en-dash
   '97' : '-',             # u'\u2014' em-dash
   '91' : "'",             # u'\u2018' left single quote
   '92' : "'",             # u'\u2019' right single quote
   '93' : '"',             # u'\u201C' left double quote
   '94' : '"',             # u'\u201D' right double quote
   '95' : "*"              # u'\u2022' bullet
}

#This is is more complex but allows for the data to be fixed.
def mixed_decoder(unicodeError):
    errStr = unicodeError[1]
    errLen = unicodeError.end - unicodeError.start
    nextPosition = unicodeError.start + errLen
    errHex = errStr[unicodeError.start:unicodeError.end].encode('hex')
    if errHex in replacement:
        return u'%s' % replacement[errHex], nextPosition
    return u'%s' % errHex, nextPosition   # Comment this line out to get a question mark
    return u'?', nextPosition

codecs.register_error("mixed", mixed_decoder)

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Talk 1 on Relevant Topic in Your Field	Talk	talk-1	UC San Francisco, Department of Testing	2012-03-01	San Francisco, California		This is a description of your talk, which is a markdown files that can be all markdown-ified like any other post. Yay markdown!
Tutorial 1 on Relevant Topic in Your Field	Tutorial	tutorial-1	UC-Berkeley Institute for Testing Science	2013-03-01	Berkeley CA, USA	http://exampleurl.com	This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
Talk 2 on Relevant Topic in Your Field	Talk	talk-2	London School of Testing	2014-02-01	London, UK	http://example2.com	This is a description of your talk, which is a markdown files that can be all markdown-ified like any other post. Yay markdown!
Conference Proceeding talk 3 on Relevant Topic in Your Field	Conference proceedings talk	talk-3	Testing Institute of

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
projects = pd.read_csv("projects.tsv", sep="\t", header=0) #,encoding = "ISO-8859-1"
projects

,title,area,url_slug,technology,guide,date,affiliation,github_repo,excerpt
0,SmartPlay,Machine Learning,smartplay,Python,NaN,27/07/2017,Microsoft Hackathon,NaN,"Offline, personalized playlist generation whic..."
1,TraKinesics,"Computer Vision, Human Computer Interaction",trakinesics,Octave/Matlab,Dr Deepti Bhatula,11/12/2014,Indian Institute of Technology Ropar,https://github.com/kbyagnik/TraKinesics,Manoeuvre screen-pointer using image processin...
2,OCR,"Machine Learning, Computer Vision",ocr,Matlab,Dr CK Narayanan,15/10/2014,Indian Institute of Technology Ropar,https://github.com/kbyagnik/OCR-MLP,Handwritten digits recognition using Back-prop...
3,AndriodDJ,"Networking, Software Engineering",androiddj,Andriod,Dr. Junghyun Jun,27/04/2015,Indian Institute of Technology Ropar,https://github.com/kbyagnik/AndroidDJ,Android app for creating a shared playlist for...
4,Lost & Found,Software Engineering,lostnfound,Django (Python),Dr Balwinder Sodhi,1/5/2014,Indian Institute of Technology Ropar,https://github.com/kbyagnik/LostNFound-master,Intranet portal for institute students to post...
5,Sudoku Solver,Artificial Intelligence,sudoku,Python,Dr CK Narayanan,22/03/2016,Indian Institute of Technology Ropar,https://github.com/kbyagnik/SudokuSolver,Solve Sudoku puzzles using constraint satisfac...
6,Private IaaS Cloud,Cloud Computing Platforms,iaascloud,"Java, VirtualBox",Dr Balwinder Sodhi,7/3/2015,Indian Institute of Technology Ropar,https://github.com/kbyagnik/IaaS-Cloud,Designed an IaaS (Infrastructure as a Service)...
7,Board visibility detector,"Internet of Things, Networking",board-visibility,"Python, Arduino",Dr. Junghyun Jun,23/09/2015,Indian Institute of Technology Ropar,NaN,Detect low visibility of text on blackboard us...
8,Amazon Price Tracker,"Data Analytics, Software Engineering",price-tracker,Scrapy (Python) MongoDB,Dr Balwinder Sodhi,7/3/2015,Indian Institute of Technology Ropar,https://github.com/kbyagnik/Amazon-PriceTracker,Analyse price trends of any product on Amazon ...
9,AmazonS4,"Data Security, Cloud Computing",amazons4,"Java, AWS",Dr Balwinder Sodhi,20/04/2015,Indian Institute of Technology Ropar,https://github.com/kbyagnik/AmazonS4,Using RSA-AES to encrypt files/folders before ...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in projects.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: projects" + "\n"
    
    if len(str(item.area)) > 3:
        md += 'area: "' + item.area + '"\n'
    else:
        md += 'area: "computer science"\n'
    
    md += "permalink: /projects/" + html_filename + "\n"
    
    if len(str(item.technology)) > 2:
        md += 'technology: "' + item.technology + '"\n'
    
    if len(str(item.date)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.guide)) > 3:
        md += "guide: " + str(item.guide) + "\n"
    else:
        md += "guide: " + str(item.guide) + "\n"
        
    if len(str(item.affiliation)) > 3:
        md += "affiliation: " + str(item.affiliation) + "\n"
        
    if len(str(item.github_repo)) > 3:
        md += 'github_repo: "' + str(item.github_repo) + '"\n'
    else:
        md += 'github_repo: "none"\n'
    
    if len(str(item.excerpt)) > 3:
        md += "excerpt: " + html_escape(str(item.excerpt)) + "\n"
    
    md += "---\n"
    
    if len(str(item.guide)) > 3:
       md += "\nGuide: " + item.guide + ",&ensp;"+ str(item.affiliation) +" \n" 
    
    if len(str(item.github_repo)) > 3:
       md += "\n[Github repository](" + item.github_repo + ")\n" 
        
    
    if len(str(item.excerpt)) > 3:
       md += "\n" + html_escape(item.excerpt) + "\n"
        
    #md = md.decode("utf-8", "mixed")
    
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with codecs.open("../_projects/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2012-03-01-talk-1.md	  2014-02-01-talk-2.md
2013-03-01-tutorial-1.md  2014-03-01-talk-3.md


In [29]:
temp = pd.read_csv("2014-lisp.md", sep="\t", header=0, error_bad_lines=False) #,encoding = "ISO-8859-1"
temp

,---
0,"title: ""LISP interpreter"""
1,collection: projects
2,"area: ""Data Structures"""
3,permalink: /projects/2/10/2014-lisp
4,"technology: ""C++"""
5,date: 2/10/2014
6,guide: Dr Apurva Mudgal
7,affiliation: Indian Institute of Technology Ropar
8,"github_repo: ""https://github.com/kbyagnik/LISP..."
9,excerpt: An interpreter for list based program...


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
